In [1]:
import os
import pandas as pd
import json
import folium

## Task 2: Unemployment rates in Switzerland

This second task caused a fair amount of confusion and a large debate over how unemployment rates are calculated in Switzerland, what is included and what is not. And it wasn't beneficial that the definitions were available only in French, Italian or German, which are not mother tongues for many.

After a prolonged data examination and reading about the attributes' meaning, we decided it would be best to first summarise our findings here and set up a common ground before proceeding with the presentation of results. We used the French version of the dataset and the definitions provided by Amstat.

### Definitions:

* <b>Taux de chômage</b> = <i>Unemployment ratio</i> -> Number of registered unemployed at the reference day (last day of the month) divided by the number of active persons, multiplied by 100.
$$ unemployment\_ratio = \frac{\#registered\_unemployed}{\#active\_persons} $$


* <b>Chômeurs inscrits</b> = <i>Registered unemployed</i> -> Persons advertised at regional employment agencies, who are unemployed and immediately available for placement. It does not matter whether they are receiving unemployment benefits or not.


* <b>Demandeurs d'emploi inscrits non-chômeurs</b> = <i>Registered employed jobseekers</i> -> Jobseekers who are registered with the regional employment offices however who, unlike the unemployed, are not immediately available for placement or have a job. The number of unemployed job-seekers is the difference between the number of registered job-seekers and the number of unemployed. They are further classified into 4 categories.
$$ registered\_employed\_jobseekers = \#registered\_unemployed\_jobseekers - \#registered\_unemployed $$


* <b>Demandeurs d'emploi inscrits</b> = <i>Registered jobseekers</i> -> All job seekers, unemployed and non-unemployed, who are registered with regional employment agencies and are looking for work.

The data we obtained from Amstat is very recent. The last available month is September 2017.

In order to have a clearer picture of the situation on the Swiss unemployment market, we decided to take a period of 13 months, from September 2016 to September 2017 and average the figures. This will show us a less biased trend over a larger period in which seasonal peaks or drops will balance. Just for comparison, we visualise the data for September 2017 separately as well, so they can be compared to the overall annual average.

In [2]:
# Load the data
raw_data = pd.read_csv('Taux-de-chomage-annuel.csv', sep=',', encoding='utf_16', header=[1, 2], skipinitialspace=True, tupleize_cols=True)
raw_data.columns = pd.MultiIndex.from_tuples(raw_data.columns)
# The dataset contains one row for each canton
raw_data.head(5)

Canton    Mois  Septembre 2016                            \
  Canton nom en francais Mesures Taux de chômage Coefficients de variation   
0                 Zurich     NaN             3.6                         A   
1                  Berne     NaN             2.6                         A   
2                Lucerne     NaN             1.9                         A   
3                    Uri     NaN             0.9                         C   
4                 Schwyz     NaN             1.6                         A   

                                                                          \
  Chômeurs inscrits Demandeurs d'emploi Demandeurs d'emploi non chômeurs   
0            29'462              36'233                            6'771   
1            14'516              19'218                            4'702   
2             4'285               7'126                            2'841   
3               179                 332                              153   
4             1'433               2'182                              749   

                                          \
  Demandeurs d'emploi non chômeurs - PET   
0                                    353   
1                                    334   
2                                    405   
3                                     35   
4                                    128   

                                             \
  Demandeurs d'emploi non chômeurs - autres   
0                                     2'392   
1                                     2'073   
2                                     1'096   
3                                        59   
4                                       326   

                                                                    \
  Demandeurs d'emploi non chômeurs - Reconversion/perfectionnement   
0                                                 89                 
1                                                113                 
2                                                231                 
3                                                  1                 
4                                                  2                 

             ...                                 Total                    \
             ...             Coefficients de variation Chômeurs inscrits   
0            ...                                     A           385'120   
1            ...                                     A           195'949   
2            ...                                     A            56'210   
3            ...                                     C             2'609   
4            ...                                     A            20'198   

                                                        \
  Demandeurs d'emploi Demandeurs d'emploi non chômeurs   
0             476'712                           91'592   
1             257'845                           61'896   
2              94'895                           38'685   
3               4'646                            2'037   
4              30'545                           10'347   

                                          \
  Demandeurs d'emploi non chômeurs - PET   
0                                  4'326   
1                                  3'896   
2                                  5'512   
3                                    403   
4                                  1'903   

                                             \
  Demandeurs d'emploi non chômeurs - autres   
0                                    33'392   
1                                    27'990   
2                                    14'756   
3                                       643   
4                                     4'673   

                                                                    \
  Demandeurs d'emploi non chômeurs - Reconversion/perfectionnement   
0                                              1'343                 
1                  

Some of the filters could have been unchecked in the first place, on the Amstat report creation website. However, we wanted to have a better insight of how the figures and attributes are related to eachother.

In [3]:
# Drop all columns that contain only NaN values
raw_data = raw_data.dropna(axis=1, how='all')

# Delete the aggregated total value at the end (last row) -> leaves only the canton values
raw_data = raw_data.drop(26, axis=0)

# Reindex the dataset -> canton names are unique, they can be used as indices
raw_data = raw_data.set_index(raw_data['Canton']['Canton nom en francais'])
raw_data.head(5)

Canton  Septembre 2016  \
                       Canton nom en francais Taux de chômage   
Canton nom en francais                                          
Zurich                                 Zurich             3.6   
Berne                                   Berne             2.6   
Lucerne                               Lucerne             1.9   
Uri                                       Uri             0.9   
Schwyz                                 Schwyz             1.6   

                                                                    \
                       Coefficients de variation Chômeurs inscrits   
Canton nom en francais                                               
Zurich                                         A            29'462   
Berne                                          A            14'516   
Lucerne                                        A             4'285   
Uri                                            C               179   
Schwyz                                         A             1'433   

                                                                             \
                       Demandeurs d'emploi Demandeurs d'emploi non chômeurs   
Canton nom en francais                                                        
Zurich                              36'233                            6'771   
Berne                               19'218                            4'702   
Lucerne                              7'126                            2'841   
Uri                                    332                              153   
Schwyz                               2'182                              749   

                                                               \
                       Demandeurs d'emploi non chômeurs - PET   
Canton nom en francais                                          
Zurich                                                    353   
Berne                                                     334   
Lucerne                                                   405   
Uri                                                        35   
Schwyz                                                    128   

                                                                  \
                       Demandeurs d'emploi non chômeurs - autres   
Canton nom en francais                                             
Zurich                                                     2'392   
Berne                                                      2'073   
Lucerne                                                    1'096   
Uri                                                           59   
Schwyz                                                       326   

                                                                                         \
                       Demandeurs d'emploi non chômeurs - Reconversion/perfectionnement   
Canton nom en francais                                                                    
Zurich                                                                 89                 
Berne                                                                 113                 
Lucerne                                                               231                 
Uri                                                                     1                 
Schwyz                                                                  2                 

                                                                              \
                       Demandeurs d'emploi non chômeurs - gain intermédiaire   
Canton nom en francais                                                         
Zurich                                                              3'937      
Berne                                                               2'182      
Lucerne                                                             1'109      
Uri                                              

In [4]:
# Take only the unemployment rates for a period of 13 months (September 2016-September 2017)
unemployment_rates = raw_data.xs(key='Taux de chômage', axis=1, level=1)
unemployment_rates.head(5)

,Septembre 2016,Octobre 2016,Novembre 2016,Décembre 2016,Janvier 2017,Février 2017,Mars 2017,Avril 2017,Mai 2017,Juin 2017,Juillet 2017,Août 2017,Septembre 2017,Total
Canton nom en francais,,,,,,,,,,,,,,
Zurich,3.6,3.6,3.7,3.8,3.9,3.9,3.8,3.6,3.5,3.4,3.4,3.4,3.3,3.6
Berne,2.6,2.6,2.8,3.0,3.0,3.0,2.9,2.7,2.6,2.4,2.4,2.5,2.4,2.7
Lucerne,1.9,1.9,2.0,2.2,2.2,2.2,2.0,1.9,1.8,1.7,1.7,1.8,1.7,1.9
Uri,0.9,1.0,1.1,1.3,1.5,1.4,1.3,1.2,1.0,0.8,0.7,0.6,0.6,1.0
Schwyz,1.6,1.7,1.8,1.9,2.0,2.0,1.9,1.8,1.7,1.6,1.7,1.7,1.7,1.8


In [5]:
# Necessary to connect the map overlay features to the dataset columns
canton_id_mappings = {
    'Zurich': 'ZH',
    'Berne': 'BE',
    'Lucerne': 'LU',
    'Uri': 'UR',
    'Schwyz': 'SZ',
    'Obwald': 'OW',
    'Nidwald': 'NW',
    'Glaris': 'GL', 
    'Zoug': 'ZG', 
    'Fribourg': 'FR', 
    'Soleure': 'SO', 
    'Bâle-Ville': 'BS', 
    'Bâle-Campagne': 'BL',
    'Schaffhouse': 'SH', 
    'Appenzell Rhodes-Extérieures': 'AR',
    'Appenzell Rhodes-Intérieures': 'AI', 
    'St-Gall': 'SG', 
    'Grisons': 'GR', 
    'Argovie': 'AG',
    'Thurgovie': 'TG', 
    'Tessin': 'TI', 
    'Vaud': 'VD', 
    'Valais': 'VS', 
    'Neuchâtel': 'NE', 
    'Genève': 'GE', 
    'Jura': 'JU'
}

# Create a pandas dataframe based on the dictionary data
mapping_df = pd.DataFrame.from_dict(canton_id_mappings, orient='index')
mapping_df.columns = ['Id']
# Merge the unemployment rates table and the canton IDs in a single dataframe
unemployment_rates = unemployment_rates.merge(mapping_df, how='outer', left_index=True, right_index=True)
unemployment_rates.head(5)

,Septembre 2016,Octobre 2016,Novembre 2016,Décembre 2016,Janvier 2017,Février 2017,Mars 2017,Avril 2017,Mai 2017,Juin 2017,Juillet 2017,Août 2017,Septembre 2017,Total,Id
Canton nom en francais,,,,,,,,,,,,,,,
Zurich,3.6,3.6,3.7,3.8,3.9,3.9,3.8,3.6,3.5,3.4,3.4,3.4,3.3,3.6,ZH
Berne,2.6,2.6,2.8,3.0,3.0,3.0,2.9,2.7,2.6,2.4,2.4,2.5,2.4,2.7,BE
Lucerne,1.9,1.9,2.0,2.2,2.2,2.2,2.0,1.9,1.8,1.7,1.7,1.8,1.7,1.9,LU
Uri,0.9,1.0,1.1,1.3,1.5,1.4,1.3,1.2,1.0,0.8,0.7,0.6,0.6,1.0,UR
Schwyz,1.6,1.7,1.8,1.9,2.0,2.0,1.9,1.8,1.7,1.6,1.7,1.7,1.7,1.8,SZ


In [6]:
switzerland_location = [46.8182, 8.2275]
geo_path = r'ch-cantons.topojson.json'
geo_json_data = json.load(open(geo_path))
    
def draw_swiss_choropleth_map(data_df, map_name, legend_caption, columns, key, filename):
    swiss_map = folium.Map(location=switzerland_location, tiles='cartodbpositron', zoom_start=8)
    
    swiss_map.choropleth(geo_data=geo_json_data, data=data_df, name=map_name, columns=columns, 
                         key_on=key, topojson='objects.cantons', fill_color='RdYlGn', 
                         fill_opacity=0.7, line_opacity=0.2, legend_name=legend_caption)
    
    folium.LayerControl().add_to(swiss_map)
    
    swiss_map.save(filename)
    return swiss_map

Annual average unemployment rate for the period September 2016-September 2017 (inclusive)

In [8]:
unemployment_map = draw_swiss_choropleth_map(unemployment_rates, 'Unemployement rates (annual average)',
                                            'Annual average unemployment rate (%)', ['Id', 'Total'], 'feature.id',
                                            'annual-unemployment-map.html')

In [9]:
from IPython.display import HTML
HTML('<iframe src="https://cdn.rawgit.com/martinj96/Ada2017/6d502387/Homework%203/annual-unemployment-map.html" height=500 width=950></iframe>')

Even though the number of bins in which the data is split is rather small, diverging colours are a good option to achieve maximal contrast. Usually a diverging colour scheme is used to emphasize a central value and how the other points or areas differ from it, which is not the case here, but this colour palette makes the map very expressive about the value of the unemployment rate for each canton, makes it very easy to compare the relative differences in unemployment rates and makes the canton borders easily differentiable.

The 26 cantons are split into 6 bins, according to their respective unemployment rate. Based on the map, the cantons of Geneva and Neuchatel have the highest annual unemployment average, as opposed to the predominantly Swiss German cantons in the East, which are very low on the scale. The majority of those have an unemployment rate of 0.9-2.5% for the given period of 13 months, with the best one being Obwald. 

The grouping of cantons into bins is done in a way that allows for a minimal difference between the values that fall into a group. There is no bin that has a larger percent point difference than 0.6 between actual data points.

The unemployment rates per canton for September 2017. Map given to serve as a comparison to the annual average rates.

In [10]:
unemployment_map_september_2017 = draw_swiss_choropleth_map(unemployment_rates, 'Unemployement rates (September 2017)',
                                            'September 2017: Unemployment rate (%)', ['Id', 'Septembre 2017'], 'feature.id',
                                            'annual-unemployment-map-september-2017.html')

In [11]:
HTML('<iframe src="https://cdn.rawgit.com/martinj96/Ada2017/6d502387/Homework%203/annual-unemployment-map-september-2017.html" height=500 width=950></iframe>')

The September 2017 results show a general drop of unemploymend rate compared to the annual average. This is visible from the changed range of values, which indicates a notable improvement. There is not a single canton whose September 2017 score is not better than the annual average. The 'worst' case is Fribourg where there is no difference, i.e. the unemployment rate in September 2017 was equal to the mean value. The highest difference can be noticed for the cantons of Valais (notice the change of colour on the map) and Neuchatel, with 0.9 and 0.7 percent points change, respectively. That now puts Geneva on the tail and Uri in the lead, with Obwald being close behind it.

#### Redefining the unemployment rate

The instructions of task 2 are somewhat misleading in terms of the definition of unemployment rate and what exactly is included in the computation formula. As we gave clear definition of the terms used in the dataset at the beginning, we will follow those to derive the data necessary for comparison between multiple potential ways of calculating the unemployment rate, including or excluding certain social groups.

Since the category of `registered unemployed` cannot be subdivided according to the definition, we use the superset `registered unemployed jobseekers` in order to show a different way of computing the ratio.

But, according to the aforementioned formula, we need the number of active persons, i.e. the active population per canton in order to compute the new rate. But since we know the rate and the number of registered unemployed per month, we can use the above formula to calculate the active population value for a particular month. In the end we simply average the 13 values to obtain the final one which will be plotted, i.e. represented on the map.

In [12]:
# Subset the original dataset keeping only the values needed to compute the new unemployment ratio
level_ind = raw_data.columns.get_level_values(1)
cols_of_interest = (level_ind == 'Taux de chômage') | (level_ind == 'Chômeurs inscrits') | (level_ind == 'Demandeurs d\'emploi') | (level_ind == 'Demandeurs d\'emploi non chômeurs')
unemployment_subset = raw_data.iloc[:, cols_of_interest].drop('Total', axis=1, level=0)
unemployment_subset.head(5)

Septembre 2016                                        \
                       Taux de chômage Chômeurs inscrits Demandeurs d'emploi   
Canton nom en francais                                                         
Zurich                             3.6            29'462              36'233   
Berne                              2.6            14'516              19'218   
Lucerne                            1.9             4'285               7'126   
Uri                                0.9               179                 332   
Schwyz                             1.6             1'433               2'182   

                                                           Octobre 2016  \
                       Demandeurs d'emploi non chômeurs Taux de chômage   
Canton nom en francais                                                    
Zurich                                            6'771             3.6   
Berne                                             4'702             2.6   
Lucerne                                           2'841             1.9   
Uri                                                 153             1.0   
Schwyz                                              749             1.7   

                                                              \
                       Chômeurs inscrits Demandeurs d'emploi   
Canton nom en francais                                         
Zurich                            29'768              36'691   
Berne                             14'888              19'836   
Lucerne                            4'294               7'246   
Uri                                  194                 343   
Schwyz                             1'474               2'247   

                                                          Novembre 2016  \
                       Demandeurs d'emploi non chômeurs Taux de chômage   
Canton nom en francais                                                    
Zurich                                            6'923             3.7   
Berne                                             4'948             2.8   
Lucerne                                           2'952             2.0   
Uri                                                 149             1.1   
Schwyz                                              773             1.8   

                                                       ...                 \
                       Chômeurs inscrits               ...                  
Canton nom en francais                                 ...                  
Zurich                            30'651               ...                  
Berne                             15'753               ...                  
Lucerne                            4'429               ...                  
Uri                                  218               ...                  
Schwyz                             1'557               ...                  

                              Juillet 2017                                   \
                       Demandeurs d'emploi Demandeurs d'emploi non chômeurs   
Canton nom en francais                                                        
Zurich                              35'112                            7'120   
Berne                               18'442                            4'809   
Lucerne                              6'805                            2'930   
Uri                                    265                              136   
Schwyz                               2'232                              785   

                             Août 2017                                        \
                       Taux de chômage Chômeurs inscrits Demandeurs d'emploi   
Canton nom en francais                                                         
Zurich                             3.4            27'514              34'542   
Berne                              2.5            13'829              18'393   
Lucerne           

In [13]:
def convert_column_to_numeric(df, level1_col, level2_col):
    for level1 in level1_col:
        for level2 in level2_col:
            # Remove the thousands separator
            df[level1, level2].replace('\'', '', regex=True, inplace=True)
            # Parse the string values into numbers
            df[level1, level2] = pd.to_numeric(df[level1, level2])
            
    return df

In [14]:
# The original dataset has a MultiIndex, therefore it is necessary to have the entire index hierarchy
level1_cols = set(unemployment_subset.columns.get_level_values(0))
level2_cols = ['Chômeurs inscrits', 'Demandeurs d\'emploi', 'Demandeurs d\'emploi non chômeurs']
convert_column_to_numeric(unemployment_subset, level1_cols, level2_cols).head(5)

Septembre 2016                                        \
                       Taux de chômage Chômeurs inscrits Demandeurs d'emploi   
Canton nom en francais                                                         
Zurich                             3.6             29462               36233   
Berne                              2.6             14516               19218   
Lucerne                            1.9              4285                7126   
Uri                                0.9               179                 332   
Schwyz                             1.6              1433                2182   

                                                           Octobre 2016  \
                       Demandeurs d'emploi non chômeurs Taux de chômage   
Canton nom en francais                                                    
Zurich                                             6771             3.6   
Berne                                              4702             2.6   
Lucerne                                            2841             1.9   
Uri                                                 153             1.0   
Schwyz                                              749             1.7   

                                                              \
                       Chômeurs inscrits Demandeurs d'emploi   
Canton nom en francais                                         
Zurich                             29768               36691   
Berne                              14888               19836   
Lucerne                             4294                7246   
Uri                                  194                 343   
Schwyz                              1474                2247   

                                                          Novembre 2016  \
                       Demandeurs d'emploi non chômeurs Taux de chômage   
Canton nom en francais                                                    
Zurich                                             6923             3.7   
Berne                                              4948             2.8   
Lucerne                                            2952             2.0   
Uri                                                 149             1.1   
Schwyz                                              773             1.8   

                                                       ...                 \
                       Chômeurs inscrits               ...                  
Canton nom en francais                                 ...                  
Zurich                             30651               ...                  
Berne                              15753               ...                  
Lucerne                             4429               ...                  
Uri                                  218               ...                  
Schwyz                              1557               ...                  

                              Juillet 2017                                   \
                       Demandeurs d'emploi Demandeurs d'emploi non chômeurs   
Canton nom en francais                                                        
Zurich                               35112                             7120   
Berne                                18442                             4809   
Lucerne                               6805                             2930   
Uri                                    265                              136   
Schwyz                                2232                              785   

                             Août 2017                                        \
                       Taux de chômage Chômeurs inscrits Demandeurs d'emploi   
Canton nom en francais                                                         
Zurich                             3.4             27514               34542   
Berne                              2.5             13829               18393   
Lucerne           

In [15]:
# Compute the new features
for col_name in level1_cols:  
    active_population = unemployment_subset[col_name, 'Chômeurs inscrits'] / unemployment_subset[col_name, 'Taux de chômage'] * 100
    unemployment_subset[col_name, 'Personnes actives'] = pd.to_numeric(round(active_population), downcast='integer')
    
    unemployment_ratio_all = unemployment_subset[col_name, 'Demandeurs d\'emploi'] / unemployment_subset[col_name, 'Personnes actives'] * 100
    unemployment_subset[col_name, 'Taux de chômage tous'] = unemployment_ratio_all
    
    unemployment_subset = unemployment_subset.sort_index(axis=1)

unemployment_subset['Total', 'Taux de chômage tous'] = unemployment_subset.loc[:, unemployment_subset.columns.get_level_values(1) == 'Taux de chômage tous'].mean(axis=1)
unemployment_subset.head(5)

Août 2017                      \
                       Chômeurs inscrits Demandeurs d'emploi   
Canton nom en francais                                         
Zurich                             27514               34542   
Berne                              13829               18393   
Lucerne                             3992                6853   
Uri                                  123                 267   
Schwyz                              1466                2240   

                                                                           \
                       Demandeurs d'emploi non chômeurs Personnes actives   
Canton nom en francais                                                      
Zurich                                             7028            809235   
Berne                                              4564            553160   
Lucerne                                            2861            221778   
Uri                                                 144             20500   
Schwyz                                              774             86235   

                                                                   Avril 2017  \
                       Taux de chômage Taux de chômage tous Chômeurs inscrits   
Canton nom en francais                                                          
Zurich                             3.4             4.268476             29542   
Berne                              2.5             3.325078             15322   
Lucerne                            1.8             3.090027              4315   
Uri                                0.6             1.302439               228   
Schwyz                             1.7             2.597553              1580   

                                                                             \
                       Demandeurs d'emploi Demandeurs d'emploi non chômeurs   
Canton nom en francais                                                        
Zurich                               36610                             7068   
Berne                                19882                             4560   
Lucerne                               7303                             2988   
Uri                                    394                              166   
Schwyz                                2389                              809   

                                                 ...           \
                       Personnes actives         ...            
Canton nom en francais                           ...            
Zurich                            820611         ...            
Berne                             567481         ...            
Lucerne                           227105         ...            
Uri                                19000         ...            
Schwyz                             87778         ...            

                          Septembre 2016                                       \
                       Personnes actives Taux de chômage Taux de chômage tous   
Canton nom en francais                                                          
Zurich                            818389             3.6             4.427357   
Berne                             558308             2.6             3.442186   
Lucerne                           225526             1.9             3.159724   
Uri                                19889             0.9             1.669264   
Schwyz                             89562             1.6             2.436301   

                          Septembre 2017                      \
                       Chômeurs inscrits Demandeurs d'emploi   
Canton nom en francais                                         
Zurich                             27225               34156   
Berne                              13658               18385   
Lucerne                             3885                6756   
Uri                                  112                 257   
S

In [16]:
# Merge the unemployment rates and the canton IDs in a single dataframe
map_data = unemployment_subset['Total'].merge(mapping_df, how='outer', left_index=True, right_index=True)
map_data.head(5)

,Taux de chômage tous,Id
Canton nom en francais,,
Zurich,4.465839,ZH
Berne,3.532383,BE
Lucerne,3.245896,LU
Uri,1.836383,UR
Schwyz,2.687822,SZ


In [17]:
unemployment_map_annual_all = draw_swiss_choropleth_map(map_data, 'Unemployement rates including employed jobseekers (annual average)',
                                            'Unemployment rate including employed jobseekers (%)', ['Id', 'Taux de chômage tous'], 'feature.id',
                                            'annual-unemployment-annual-all.html')

In [18]:
HTML('<iframe src="https://cdn.rawgit.com/martinj96/Ada2017/e78fe89b/Homework%203/annual-unemployment-annual-all.html" height=500 width=950></iframe>')

Naturally, when adding a particular group of people such as the registered employed jobseekers to the definition of unemployed people, the rate goes up in absolute terms, as expected. However, the majority of the cantons doesn't change significantly change their rank. For most of them the new calculation method of the rate just introduced a certain shift. Observing the data, the largest 'jump' in the values can be noticed for Fribourg, Tessin (Ticino) and Jura, with over 2 percent points difference.

This map representation shows a tendency of the Eastern cantons towards the overall average, which was not the case for the annual average rate computed previously, where they were among the best in terms of unemployment rate. The relative position of the French-speaking cantons and Tessin (Ticino). The most notably high unemployment rates are still observed in Geneva and Neuchatel.